In [1]:
import os
import pandas as pd
import numpy as np
import random
import itertools

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

import sys
sys.path.append('/data/Hydra_Work/Competition_Functions') 
from Processing_Functions import process_forecast_date, process_seasonal_forecasts
from Data_Transforming import read_nested_csvs, generate_daily_flow, use_USGS_flow_data, USGS_to_daily_df_yearly


sys.path.append('/data/Hydra_Work/Pipeline_Functions')
from Folder_Work import filter_rows_by_year, csv_dictionary, add_day_of_year_column

from datetime import datetime

In [2]:
# All the prep
monthly_basins = ['animas_r_at_durango', 'boise_r_nr_boise', 'boysen_reservoir_inflow', 'colville_r_at_kettle_falls', 'detroit_lake_inflow', 'dillon_reservoir_inflow',
    'fontenelle_reservoir_inflow', 'green_r_bl_howard_a_hanson_dam', 'hungry_horse_reservoir_inflow', 'libby_reservoir_inflow',
    'missouri_r_at_toston','owyhee_r_bl_owyhee_dam', 'pecos_r_nr_pecos', 'pueblo_reservoir_inflow',
    'ruedi_reservoir_inflow', 'skagit_ross_reservoir', 'snake_r_nr_heise', 'stehekin_r_at_stehekin', 'sweetwater_r_nr_alcova',
    'taylor_park_reservoir_inflow', 'virgin_r_at_virtin', 'weber_r_nr_oakley', 'yampa_r_nr_maybell',
]


USGS_basins = ['animas_r_at_durango', 'boise_r_nr_boise', 'boysen_reservoir_inflow', 'colville_r_at_kettle_falls', 'detroit_lake_inflow', 'dillon_reservoir_inflow',   
    'green_r_bl_howard_a_hanson_dam', 'hungry_horse_reservoir_inflow', 'libby_reservoir_inflow', 'merced_river_yosemite_at_pohono_bridge', 'missouri_r_at_toston',
    'owyhee_r_bl_owyhee_dam', 'pecos_r_nr_pecos', 'pueblo_reservoir_inflow',    'san_joaquin_river_millerton_reservoir', 'snake_r_nr_heise', 'stehekin_r_at_stehekin',
    'sweetwater_r_nr_alcova', 'taylor_park_reservoir_inflow', 'virgin_r_at_virtin', 'weber_r_nr_oakley', 'yampa_r_nr_maybell',
]

basins = list(set(monthly_basins + USGS_basins))


selected_years = range(2000,2024,2)

era5_folder = '/data/Hydra_Work/Rodeo_Data/era5'
era5 = csv_dictionary(era5_folder, basins, years=selected_years)



In [3]:
# Initialize an empty dictionary to store aggregated dataframes
aggregated_year_era5 = {}

# Iterate through each dataframe in the dictionary
for key, df in era5.items():
    # Extract basin name by ignoring the last 4 characters (assuming year is always the last 4 characters)
    basin = key[:-4]
    
    # Check if basin already exists in the aggregated dictionary
    if basin in aggregated_year_era5:
        # If it exists, append the dataframe to the existing one
        aggregated_year_era5[basin] = pd.concat([aggregated_year_era5[basin], df])
    else:
        # If it doesn't exist, add the dataframe to the aggregated dictionary
        aggregated_year_era5[basin] = df

In [4]:
# Initialize an empty dataframe to store aggregated statistics
Catchment_Summaries = pd.DataFrame()

# Iterate through each basin dataframe in the aggregated dictionary
for basin, df in aggregated_year_era5.items():
    # Calculate mean, min, and max for each variable
    means = df.mean()
    mins = df.min()
    maxs = df.max()
    
    # Concatenate mean, min, and max into one dataframe
    stats_df = pd.concat([means], axis=1)
    stats_df.columns = ['Mean']  # Rename columns
    
    # Add a column for basin name
    stats_df['Basin'] = basin
    
    # Append to the aggregated dataframe
    Catchment_Summaries = pd.concat([Catchment_Summaries, stats_df])

# Reset index of the aggregated dataframe
Catchment_Summaries.reset_index(inplace=True)



In [5]:
Catchment_Summaries.set_index('Basin', inplace=True)
Catchment_Summaries.rename(columns={'index': 'Variable'}, inplace=True)
Catchment_Summaries = Catchment_Summaries.pivot(columns='Variable')

# Flatten the multi-index column
Catchment_Summaries.columns = [f'{col[1]}_{col[0]}' for col in Catchment_Summaries.columns]


In [6]:
Catchment_Summaries.max()

evaporation_Mean                     -0.003424
precipitation_Mean                    0.070088
runoff_Mean                           0.054978
snow depth water equivalent_Mean      0.572583
soil water volume_Mean                0.382638
temperature_Mean                    282.339275
dtype: float64

In [24]:
Grace = pd.read_csv('/data/Hydra_Work/Rodeo_Data/static_data/grace_aggregated.csv')
Grace.drop(columns=['time'], inplace=True)


Static = pd.read_csv('/data/Hydra_Work/Rodeo_Data/static_indices.csv')


In [29]:
Static_Grace = Grace.groupby('site_id').mean()
Static_Grace.reset_index(inplace = True)

In [31]:
Static = pd.merge(Static, Static_Grace, on='site_id', how='inner')


In [35]:
Static.to_csv('/data/Hydra_Work/Rodeo_Data/static_indices.csv', index = False)

In [36]:
pd.read_csv('/data/Hydra_Work/Rodeo_Data/static_indices.csv')

,site_id,ele_min,ele_max,longitude_mean,latitude_mean,ele_mean,area,mean_gws_inst,mean_rtzsm_inst,mean_sfsm_inst
0,sweetwater_r_nr_alcova,1791.595300,3705.0945,-108.021964,42.471149,2158.377059,2377.280,47.650540,48.823513,50.111801
1,dillon_reservoir_inflow,2684.053200,4240.8696,-106.034084,39.520124,3340.723288,328.429,36.410120,38.318838,40.610818
2,boise_r_nr_boise,887.193540,3068.0050,-115.414110,43.692290,1827.530677,2687.340,27.136775,30.778627,33.640614
3,animas_r_at_durango,1992.508900,4124.5063,-107.780574,37.623373,3106.410076,700.901,31.110382,35.419485,37.207589
4,owyhee_r_bl_owyhee_dam,734.171140,3103.0300,-117.096725,42.438306,1565.652117,11659.800,41.276150,45.958140,47.287702
5,virgin_r_at_virtin,1074.136600,3133.7050,-112.871350,37.304299,1963.739747,948.039,33.353461,40.920170,40.592804
6,yampa_r_nr_maybell,1801.979200,3691.5610,-107.286375,40.445881,2365.734470,3381.680,34.417273,36.779511,38.458430
7,hungry_horse_reservoir_inflow,970.679750,2694.2378,-113.456741,47.783586,1779.857616,1681.780,24.506489,27.554974,30.462586
8,san_joaquin_river_millerton_reservoir,63.953293,3007.7756,-120.583297,38.941244,1336.659923,1886.720,39.556052,41.275360,42.149375
9,american_river_folsom_lake,96.057280,4063.4822,-119.220857,37.331104,2107.505785,1677.530,32.513724,36.180342,36.427954
